In [ ]:
import pandas as pd
import numpy as np
#import autograd
import random

In [ ]:
df = pd.read_csv('nice_data.csv')

In [ ]:
subjects = list(df.subject.unique())
reviewers = list(df.reviewer.unique())
rm = df.score.mean()
rs = df.score.std()

n_sub = len(subjects)
n_rev = len(reviewers)
dataset = np.array(df)
total_reviews = np.zeros(n_sub)
num_reviews = np.zeros(n_sub)



for i in range(dataset.shape[0]):
    dataset[i,0] = subjects.index(dataset[i,0])
    dataset[i,1] = reviewers.index(dataset[i,1])
    #dataset[i,2] = (dataset[i,2]-rm)/rs
    
    # normalization step
    total_reviews[dataset[i,0]] += dataset[i,2]
    num_reviews[dataset[i,0]] += 1

average_reviews = total_reviews/num_reviews
np.random.shuffle(dataset)

In [ ]:
reviewer_means = np.zeros(n_rev)
reviewer_std = np.zeros(n_rev)
for idx,reviewer in enumerate(reviewers):
    reviewer_means[idx] = df[df.reviewer == reviewer].score.mean()
    reviewer_std[idx] = df[df.reviewer == reviewer].score.std()

In [ ]:
reviewer_biases = np.random.randn(n_rev)
subject_scores = np.random.random(n_sub)
reg_l2= 1/2
lr = 1e-9
num_iter = 5000
x0 = np.hstack([reviewer_biases,subject_scores])

In [ ]:
def loss(x):
    res = 0
    rb = x[:n_rev]
    ss = x[n_rev:]
    #d = dataset[np.random.randint(dataset.shape[0])]
    #res = res + (d[2]-rb[d[1]]-ss[d[0]])**2 + reg_l2* (rb**2).sum()
    for d in dataset:
        res = res + (d[2]-rb[d[1]]-ss[d[0]])**2
    res = res+ reg_l2* (rb**2).sum()
    return res
loss(x0)

In [ ]:
if False:
    J = autograd.jacobian(loss)
    x = x0
    for i in range(50):
        g = J(x)
        l = loss(x)
        x = x - lr * g
        #x = x - lr/pow(i+1,0.5) * g
        if (i % 2 == 0):
            #print(l,sum([loss(x) for _ in range(100)]))
            print(l)
        if(i % 10 == 0):
            lr = lr/4
else:
    import scipy.optimize as opt
    res = opt.minimize(loss,x0)
    x = res.x

In [ ]:
res

In [ ]:
reviewer_biases = x[:n_rev]
subject_scores = x[n_rev:]

In [ ]:
reviewer_biases

In [ ]:
print('{}\t\t{}'.format('name','bias'))

for i in np.argsort(reviewer_biases):
    print('{}\t\t{:.2f}'.format(reviewers[i],reviewer_biases[i]))

In [ ]:
print('{}\t\t{}\t{}'.format('name','mean','std'))
for i in np.argsort(reviewer_means):
    print('{}\t\t{:.2f}\t{:.2f}'.format(reviewers[i],reviewer_means[i],reviewer_std[i]))

In [ ]:
subject_scores

In [ ]:
average_reviews

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(8,8))
#plt.scatter(average_reviews,subject_scores+(average_reviews.mean()-subject_scores.mean()))
plt.scatter(average_reviews,subject_scores)
plt.xlabel('average review')
plt.ylabel('true review')
maxv = max(abs(average_reviews).max(),abs(subject_scores).max())
#plt.xlim(-maxv,maxv)
#plt.ylim(-maxv,maxv)
plt.grid(True)
plt.show()